# Needleman–Wunsch algorithm 
For string alignment.

Given the cost of `gap`, `match` and `mismatch` calculate the matrices S (score) and T (trace) that give the minimum alignment cost.

In the **trace** matrix we have the following direction encoding:
 * 1 - Diagonal (down and right)
 * 2 - Vertical (down)
 * 3 - Horizontal (ltr)

In [15]:
import numpy as np

In [96]:
TERMINATION = 0
DIAGONAL    = 1
VERTICAL    = 2
HORIZONTAL  = 3

In [98]:
seq1 = "ACTA"
seq2 = "TACT"

In [142]:
def needleman_wunsch(s1, s2, gap=-3, match=3, mismatch=-1):
    dim = (len(s1)+1, len(s2)+1)
    S, T = np.zeros(shape=dim, dtype=int), np.zeros(shape=dim, dtype=int)
    # initial values
    T[0]  = HORIZONTAL
    T[:,0]= VERTICAL
    T[0,0]= TERMINATION
    S[0]   = [i * gap for i in range(len(s1)+1)]
    S[:,0] = [i * gap for i in range(len(s2)+1)]
    # perform algoritm
    T = T.tolist()
    for i, c1 in enumerate(s1):
        for j, c2 in enumerate(s2):
            cost = match if c1 == c2 else mismatch
            scores = [S[i][j] + cost, S[i+1][j] + gap, S[i][j+1] + gap]
            m = max(scores)
            S[i+1][j+1] = m
            T[i+1][j+1] = [i+1 for i,x in enumerate(scores) if x == m]
    for i in range(dim[0]):T[0][i] = [T[0][i]]
    for i in range(dim[1]):T[i][0] = [T[i][0]]
    return S, T

In [143]:
print(needleman_wunsch(seq1, seq2))

(array([[  0,  -3,  -6,  -9, -12],
       [ -3,  -1,   0,  -3,  -6],
       [ -6,  -4,  -2,   3,   0],
       [ -9,  -3,  -5,   0,   6],
       [-12,  -6,   0,  -3,   3]]), [[[[0]], [3], [3], [3], [3]], [[2], [1], [1], [2], [2]], [[2], [1, 3], [1], [1], [2]], [[2], [1], [1, 3], [3], [1]], [[2], [3], [1], [2, 3], [3]]])


In [153]:
def backtrack_needleman_wunsch(s1, s2, S, T):
    i, j = map(lambda x: x-1, S.shape)
    res = ["", ""]
    while i > 0 or j > 0:
        if DIAGONAL in T[i][j]:
            res[0]+=s1[-1]; s1 = s1[:-1]
            res[1]+=s2[-1]; s2 = s2[:-1]
            i-=1
            j-=1
        elif HORIZONTAL in T[i][j]:
            res[0]+=s1[-1]; s1 = s1[:-1]
            res[1]+="_"
            j-=1
        elif VERTICAL in T[i][j]:
            res[0]+="_"
            res[1]+=s2[-1]; s2 = s2[:-1]
            i-=1
        
    return tuple(map(lambda x: x[::-1], res))

In [154]:
s, t = needleman_wunsch(seq1, seq2)
backtrack_needleman_wunsch(seq1, seq2, s, t)

('__ACTA', 'TACT__')

# Types of Alignment
There are 2 types of alignment, local and global:
* local  - por exemplo quando duas espécies já divergiram muito mas ainda podem ter partes em comum
* global - se se espera uma similariedade grande entre a sequência

# Smith Waterman - Local sequence alignment
Reformulates the notion of sequence in the DP algorithm

The S matrix is initialized with 0s on the first row and column.

If the result of a cell is negative, the actual value to use is 0.

To backtrack we find the largest